## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
from ipywidgets import interact, interactive, fixed
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import os.path
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import sys
sys.path.append('../')
from utils import build_distort_p, load_distort_p, get_perspective_transform, s_filter, sobel_filter, combine_binaries, draw_result
from showcase import show_calibration_result, show_perspective_transform_result, show_color_threshold_result, show_filtered_warped_image
from poly import fit_poly, fit_poly_vis

In [ ]:
DISTORT_P = 'distort.p'

is_show_calibration_result = False
is_show_perspective_transform_result = False
is_show_color_threshold_result = False
is_show_filtered_warped_image = False

## First, I'll compute the camera calibration using chessboard images

In [ ]:
if not os.path.isfile(DISTORT_P):
    build_distort_p(DISTORT_P)

mtx, dist = load_distort_p(DISTORT_P)

In [ ]:
if is_show_calibration_result:
    show_calibration_result(mtx, dist)

## Decide the perspective tranform matrix

In [ ]:
M, Minv = get_perspective_transform()

In [ ]:
if is_show_perspective_transform_result:
    show_perspective_transform_result(mtx, dist, M)

## Use color transforms, gradients, etc., to create a thresholded binary image

1. Color space (Gray, HLS, YUV, HSV)
2. Sobel
3. Magnitude of the Gradient


In [ ]:
if is_show_color_threshold_result:
    show_color_threshold_result(mtx, dist, M)

In [ ]:
if is_show_filtered_warped_image:
    show_filtered_warped_image(mtx, dist, M)

In [ ]:
images = glob.glob('../test_images/*.jpg')

for fname in images:
    img = mpimg.imread(fname)

    undist = cv2.undistort(img, mtx, dist, None, mtx)

    s_binary = s_filter(undist)
    sxbinary = sobel_filter(undist)
    combined_binary = combine_binaries(s_binary, sxbinary)

    binary_warped = cv2.warpPerspective(combined_binary, M, (combined_binary.shape[1], combined_binary.shape[0]), flags=cv2.INTER_LINEAR)

    left_fit, right_fit = fit_poly(binary_warped)
    
    fit_poly_vis(binary_warped, left_fit, right_fit)

#     print(left_fit, right_fit)
    
#     fig = plt.figure(figsize=(20, 12))

#     plt.subplot(121)
#     plt.imshow(combined_binary, cmap='gray')
#     plt.title('combined ('+fname+')')

#     plt.subplot(122)
#     plt.imshow(warped, cmap='gray')
#     plt.title('warped')

#     plt.axis('off')
#     fig.tight_layout()
#     plt.show()

In [ ]:
import imageio
imageio.plugins.ffmpeg.download()

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# Define global variables here

In [ ]:
def process_image(image):
    undist = cv2.undistort(image, mtx, dist, None, mtx)

    s_binary = s_filter(undist)
    sxbinary = sobel_filter(undist)
    combined_binary = combine_binaries(s_binary, sxbinary)

    binary_warped = cv2.warpPerspective(combined_binary, M, (combined_binary.shape[1], combined_binary.shape[0]), flags=cv2.INTER_LINEAR)

    left_fit, right_fit = fit_poly(binary_warped)

    overlay_img = draw_result(undist, binary_warped, left_fit, right_fit, Minv)

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(overlay_img,'Radius of Curvature = 744(m)',(20,60), font, 2, (255,255,255),2,cv2.LINE_AA)
    cv2.putText(overlay_img,'Vehicle is 0.21m left of center',(20,120), font, 2, (255,255,255),2,cv2.LINE_AA)
    
    return overlay_img

In [ ]:
output = 'output.mp4'
clip1 = VideoFileClip('../project_video.mp4')
clip = clip1.fl_image(process_image)
%time clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))